# HAI-22.04 Dataset Anomaly Detection - PCA and Mahalanobis Distance

This notebook uses Principal Component Analysis (PCA) and Mahalanobis Distance for anomaly detection on the HAI-22.04 dataset.

The HAI dataset contains data from industrial control systems (ICS), where training data does not include attack labels, while test data includes attack labels.

## 1. Import Necessary Libraries

In [ ]:
# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import pickle
import time
from scipy.stats import chi2

# Import machine learning libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EmpiricalCovariance
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, roc_curve, f1_score, precision_score, recall_score

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Set random seed for reproducibility
np.random.seed(42)

## 2. Load Dataset

In [ ]:
# Set data path
data_path = "../../hai-security-dataset/hai-22.04/"

# Load training data
train_files = [f for f in os.listdir(data_path) if f.startswith('train')]
train_dfs = []

for file in train_files:
    print(f"Loading training file: {file}")
    df = pd.read_csv(f"{data_path}{file}")
    train_dfs.append(df)
    
# Combine training data
train_df = pd.concat(train_dfs, ignore_index=True)
print(f"Training data shape: {train_df.shape}")

In [ ]:
# Load test data
test_files = [f for f in os.listdir(data_path) if f.startswith('test')]
test_dfs = []

for file in test_files:
    print(f"Loading test file: {file}")
    df = pd.read_csv(f"{data_path}{file}")
    test_dfs.append(df)
    
# Combine test data
test_df = pd.concat(test_dfs, ignore_index=True)
print(f"Test data shape: {test_df.shape}")

## 3. Data Preprocessing

In [ ]:
# Check basic information of the dataset
print("Column names in training dataset:")
print(train_df.columns.tolist())

# Check for missing values
print("\nMissing values in training dataset:")
print(train_df.isnull().sum().sum())

print("\nMissing values in test dataset:")
print(test_df.isnull().sum().sum())

In [ ]:
# Convert timestamp to datetime
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

# Extract attack labels from test data
attack_columns = [col for col in test_df.columns if 'attack' in col.lower()]
print(f"Attack label columns: {attack_columns}")

# Create a combined attack label (if multiple attack columns exist)
if len(attack_columns) > 1:
    test_df['attack_combined'] = test_df[attack_columns].max(axis=1)
    y_test = test_df['attack_combined']
else:
    y_test = test_df[attack_columns[0]]

# Print attack distribution
print(f"\nAttack distribution in test data:\n{y_test.value_counts()}")

In [ ]:
# Select features for training and testing
# Exclude timestamp and attack labels
feature_columns = [col for col in train_df.columns if col not in ['timestamp'] + attack_columns]
print(f"Number of features: {len(feature_columns)}")

# Prepare training and testing data
X_train = train_df[feature_columns].values
X_test = test_df[feature_columns].values

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training data shape after preprocessing: {X_train_scaled.shape}")
print(f"Testing data shape after preprocessing: {X_test_scaled.shape}")

## 4. Apply Principal Component Analysis (PCA)

In [ ]:
# Determine the number of components to retain
# We'll use a variance threshold approach

# First, fit PCA with all components
pca_all = PCA()
pca_all.fit(X_train_scaled)

# Plot explained variance ratio
plt.figure(figsize=(12, 6))
plt.plot(np.cumsum(pca_all.explained_variance_ratio_), marker='o')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.axhline(y=0.99, color='g', linestyle='--', label='99% Explained Variance')
plt.title('Cumulative Explained Variance Ratio')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.legend()
plt.show()

# Find number of components for 95% and 99% variance
n_components_95 = np.argmax(np.cumsum(pca_all.explained_variance_ratio_) >= 0.95) + 1
n_components_99 = np.argmax(np.cumsum(pca_all.explained_variance_ratio_) >= 0.99) + 1

print(f"Number of components for 95% variance: {n_components_95}")
print(f"Number of components for 99% variance: {n_components_99}")

In [ ]:
# We'll use the number of components for 95% variance
n_components = n_components_95

# Apply PCA with the selected number of components
print(f"Applying PCA with {n_components} components...")
start_time = time.time()

pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

pca_time = time.time() - start_time
print(f"PCA completed in {pca_time:.2f} seconds")

print(f"PCA transformed training data shape: {X_train_pca.shape}")
print(f"PCA transformed testing data shape: {X_test_pca.shape}")

## 5. Calculate Mahalanobis Distance

In [ ]:
# Estimate the covariance matrix from the PCA-transformed training data
print("Estimating covariance matrix...")
start_time = time.time()

cov = EmpiricalCovariance().fit(X_train_pca)

cov_time = time.time() - start_time
print(f"Covariance estimation completed in {cov_time:.2f} seconds")

# Calculate the mean of the PCA-transformed training data
mean_vec = np.mean(X_train_pca, axis=0)

# Function to calculate Mahalanobis distance
def mahalanobis_distance(x, mean, cov):
    inv_cov = np.linalg.inv(cov.covariance_)
    x_minus_mean = x - mean
    left = np.dot(x_minus_mean, inv_cov)
    mahal = np.dot(left, x_minus_mean.T)
    return np.sqrt(mahal.diagonal())

# Calculate Mahalanobis distance for training and test data
print("Calculating Mahalanobis distances...")
start_time = time.time()

train_mahal = mahalanobis_distance(X_train_pca, mean_vec, cov)
test_mahal = mahalanobis_distance(X_test_pca, mean_vec, cov)

mahal_time = time.time() - start_time
print(f"Mahalanobis distance calculation completed in {mahal_time:.2f} seconds")

In [ ]:
# Plot Mahalanobis distance distribution for training data
plt.figure(figsize=(12, 6))
plt.hist(train_mahal, bins=50, alpha=0.7, label='Training Data')
plt.title('Mahalanobis Distance Distribution (Training Data)')
plt.xlabel('Mahalanobis Distance')
plt.ylabel('Frequency')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Plot Mahalanobis distance distribution for test data by class
plt.figure(figsize=(12, 6))
plt.hist(test_mahal[y_test == 0], bins=50, alpha=0.7, label='Normal', color='blue')
plt.hist(test_mahal[y_test == 1], bins=50, alpha=0.7, label='Anomaly', color='red')
plt.title('Mahalanobis Distance by Class (Test Data)')
plt.xlabel('Mahalanobis Distance')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

## 6. Save the Model Components

In [ ]:
# Create a directory to save the model if it doesn't exist
model_dir = "./"
os.makedirs(model_dir, exist_ok=True)

# Save the model components
scaler_filename = os.path.join(model_dir, "scaler.pkl")
pca_filename = os.path.join(model_dir, "pca_model.pkl")
cov_filename = os.path.join(model_dir, "covariance_model.pkl")
mean_filename = os.path.join(model_dir, "mean_vector.pkl")

with open(scaler_filename, 'wb') as file:
    pickle.dump(scaler, file)
    
with open(pca_filename, 'wb') as file:
    pickle.dump(pca, file)
    
with open(cov_filename, 'wb') as file:
    pickle.dump(cov, file)
    
with open(mean_filename, 'wb') as file:
    pickle.dump(mean_vec, file)
    
print(f"Scaler saved to {scaler_filename}")
print(f"PCA model saved to {pca_filename}")
print(f"Covariance model saved to {cov_filename}")
print(f"Mean vector saved to {mean_filename}")

## 7. Threshold Optimization

In [ ]:
# Try different thresholds for anomaly detection
thresholds = np.linspace(min(test_mahal), max(test_mahal), 100)
f1_scores = []
precision_scores = []
recall_scores = []

for threshold in thresholds:
    y_pred_threshold = np.where(test_mahal >= threshold, 1, 0)
    f1_scores.append(f1_score(y_test, y_pred_threshold))
    precision_scores.append(precision_score(y_test, y_pred_threshold))
    recall_scores.append(recall_score(y_test, y_pred_threshold))

# Find the threshold that maximizes F1 score
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]
best_f1 = f1_scores[best_threshold_idx]
best_precision = precision_scores[best_threshold_idx]
best_recall = recall_scores[best_threshold_idx]

print(f"Best threshold: {best_threshold:.4f}")
print(f"Best F1 score: {best_f1:.4f}")
print(f"Precision at best threshold: {best_precision:.4f}")
print(f"Recall at best threshold: {best_recall:.4f}")

In [ ]:
# Plot F1, precision, and recall scores for different thresholds
plt.figure(figsize=(12, 8))
plt.plot(thresholds, f1_scores, 'b-', label='F1 Score')
plt.plot(thresholds, precision_scores, 'g-', label='Precision')
plt.plot(thresholds, recall_scores, 'r-', label='Recall')
plt.axvline(x=best_threshold, color='k', linestyle='--', label=f'Best Threshold: {best_threshold:.4f}')
plt.title('Performance Metrics vs. Threshold')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compare with theoretical threshold based on chi-square distribution
# For Mahalanobis distance squared, the threshold can be derived from chi-square distribution
# with degrees of freedom equal to the number of components

# Calculate theoretical thresholds for different confidence levels
confidence_levels = [0.90, 0.95, 0.99, 0.999]
theoretical_thresholds = [np.sqrt(chi2.ppf(cl, n_components)) for cl in confidence_levels]

print("Theoretical thresholds based on chi-square distribution:")
for cl, th in zip(confidence_levels, theoretical_thresholds):
    print(f"Confidence level {cl*100:.1f}%: {th:.4f}")

print(f"\nEmpirical best threshold: {best_threshold:.4f}")

In [ ]:
# Save the optimized threshold
threshold_filename = os.path.join(model_dir, "optimized_threshold.pkl")
with open(threshold_filename, 'wb') as file:
    pickle.dump(best_threshold, file)
    
print(f"Optimized threshold saved to {threshold_filename}")

## 8. Evaluate the Model

In [ ]:
# Apply the optimized threshold
y_pred_optimized = np.where(test_mahal >= best_threshold, 1, 0)

# Evaluate with optimized threshold
print("Confusion Matrix with Optimized Threshold:")
cm_optimized = confusion_matrix(y_test, y_pred_optimized)
print(cm_optimized)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_optimized, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Normal', 'Anomaly'],
            yticklabels=['Normal', 'Anomaly'])
plt.title('Confusion Matrix with Optimized Threshold')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Classification report
print("\nClassification Report with Optimized Threshold:")
print(classification_report(y_test, y_pred_optimized))

In [ ]:
# ROC Curve
fpr, tpr, _ = roc_curve(y_test, test_mahal)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Precision-Recall Curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test, test_mahal)
pr_auc = auc(recall_curve, precision_curve)

plt.figure(figsize=(10, 8))
plt.plot(recall_curve, precision_curve, color='blue', lw=2, label=f'PR curve (area = {pr_auc:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

## 9. Analyze Anomalies Over Time

In [ ]:
# Create a DataFrame with timestamps, actual labels, and predictions
results_df = pd.DataFrame({
    'timestamp': test_df['timestamp'],
    'actual': y_test,
    'predicted': y_pred_optimized,
    'mahalanobis_distance': test_mahal
})

# Plot actual vs predicted anomalies over time
plt.figure(figsize=(16, 8))

# Sample data for better visualization if dataset is large
sample_size = min(10000, len(results_df))
sample_indices = np.linspace(0, len(results_df)-1, sample_size, dtype=int)
sample_df = results_df.iloc[sample_indices]

plt.plot(sample_df['timestamp'], sample_df['actual'], 'b-', alpha=0.5, label='Actual')
plt.plot(sample_df['timestamp'], sample_df['predicted'], 'r-', alpha=0.5, label='Predicted')
plt.title('Actual vs Predicted Anomalies Over Time')
plt.xlabel('Time')
plt.ylabel('Anomaly (1) / Normal (0)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot Mahalanobis distance over time with actual labels
plt.figure(figsize=(16, 8))

# Create a colormap based on actual labels
colors = np.where(sample_df['actual'] == 1, 'red', 'blue')

plt.scatter(sample_df['timestamp'], sample_df['mahalanobis_distance'], c=colors, alpha=0.5, s=10)
plt.title('Mahalanobis Distance Over Time')
plt.xlabel('Time')
plt.ylabel('Mahalanobis Distance')
plt.grid(True)

# Add a horizontal line at the threshold
plt.axhline(y=best_threshold, color='g', linestyle='--')

# Add a legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Actual Anomaly'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Normal'),
    Line2D([0], [0], color='g', linestyle='--', label='Threshold')
]
plt.legend(handles=legend_elements)

plt.show()

## 10. Feature Importance Analysis

In [ ]:
# Analyze feature importance based on PCA components
# Get the absolute values of the PCA components
component_importance = np.abs(pca.components_)

# Sum the importance across all components, weighted by explained variance
feature_importance = np.sum(component_importance.T * pca.explained_variance_ratio_, axis=1)

# Normalize to get relative importance
feature_importance = feature_importance / np.sum(feature_importance)

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importance
})

# Sort by importance
importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

# Display the top 20 most important features
print("Top 20 most important features:")
importance_df.head(20)

In [ ]:
# Plot feature importance
plt.figure(figsize=(14, 10))
top_n = 20  # Show top 20 features
sns.barplot(x='Importance', y='Feature', data=importance_df.head(top_n))
plt.title(f'Top {top_n} Feature Importance Based on PCA')
plt.xlabel('Relative Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## 11. Conclusion

In this notebook, we have:

1. Loaded and preprocessed the HAI-22.04 dataset
2. Applied Principal Component Analysis (PCA) for dimensionality reduction
3. Calculated Mahalanobis distances to detect anomalies
4. Optimized the anomaly detection threshold to maximize F1 score
5. Evaluated the model's performance using various metrics
6. Analyzed feature importance based on PCA components
7. Saved the model components and optimized threshold for future use

The combination of PCA and Mahalanobis distance has demonstrated its effectiveness for anomaly detection in industrial control system data. This approach has several advantages:

- It reduces the dimensionality of the data while preserving most of the variance
- It takes into account the correlation between features
- It provides a statistical foundation for setting anomaly thresholds
- It is computationally efficient for high-dimensional data

This method is particularly suitable for industrial control systems where normal operation follows a multivariate Gaussian distribution, and anomalies represent deviations from this distribution.